# Import

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

# Generating linearly separable data

In [ ]:
n = 8000
d = 100
tau = 0.1

a = np.random.rand(d-1,1)
a = a / np.sum(a)

X = np.random.rand(n,d - 1) - 0.5
X0 = -np.dot(X[:int(n/2)], a) + np.random.rand(int(n/2),1) - tau
X1 = -np.dot(X[int(n/2):], a) + np.random.rand(int(n/2),1) + (tau + 1)
Xd = np.concatenate((X0, X1), axis=0) 
X = np.concatenate((X, Xd), axis=1)
Y = np.concatenate((-np.ones((int(n/2),1)), np.ones((int(n/2),1))), axis=0)

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
plt.plot(-np.dot(X[:int(n/2),:-1], a), X[:int(n/2), -1], 'r.')
plt.plot(-np.dot(X[int(n/2):,:-1], a), X[int(n/2):, -1], 'b.')
plt.show()

In [ ]:
# Compute L
print(np.matmul(X.T,X).shape)
eigens = np.linalg.eigh(np.matmul(X.T,X))[0]
L = eigens[-1] # The eigenvalues in ascending order
print(L)

In [ ]:
epochs = 50

# SGD

In [ ]:
def SGD(X, Y, epochs, lr):
    n, d = X.shape
    w = np.zeros((d,1))
    b = 0
    losses = []
    accs = []
    # epoch 0
    z = np.dot(X, w) + b
    pred = z > 0
    pred = pred * 2 - 1
    acc = np.mean(pred == Y)
    accs.append(acc)
    loss = np.mean(np.maximum(0, 1 - Y* z))
    losses.append(loss)
    
    cnt = 0 
    for epoch in range(epochs):
        for i in range(n):
            idx = np.random.randint(n)
            x = X[idx]
            x = np.expand_dims(x, axis=-1)
            y = Y[idx]
            J_w = 0
            J_b = 0
            if y*(np.dot(w.T,x) + b) < 1:    
                J_w = -y*x
                J_b = -y
                cnt += 1
            w = w - lr*J_w
            b = b - lr*J_b
            
        z = np.dot(X, w) + b
        pred = z > 0
        pred = pred * 2 - 1
        acc = np.mean(pred == Y)
        accs.append(acc)
        loss = np.mean(np.maximum(0, 1 - Y* z))
        losses.append(loss)
        
    return losses, accs

In [ ]:
sgd_losses, sgd_accs = SGD(X, Y, epochs, 1/L)
print(sgd_losses)
print(sgd_accs)

In [ ]:
plt.plot(range(epochs + 1), sgd_losses)
plt.yscale('log')
plt.legend(['SGD loss'])

# Acc-SGD

In [ ]:
def Acc_SGD(X, Y, epochs, rho, tau):
    eta = tau / L
    n, d = X.shape
    X = np.concatenate((X, np.ones((n,1))), axis=1)
    w = np.zeros((d+1,1))
    b = 0
    losses = []
    accs = []
    # update param
    v = np.zeros((d+1,1))
    zeta = np.zeros((d+1,1))
    gamma = 0
    ak = 0
    beta = 1 # set mu = 0 (0-strong convex function is a convex function)
    
    # epoch 0
    z = np.dot(X, w) + b
    pred = z > 0
    pred = pred * 2 - 1
    acc = np.mean(pred == Y)
    accs.append(acc)
    loss = np.mean(np.maximum(0, 1 - Y* z)) # hinge loss for SVM
    losses.append(loss)
    
    for epoch in range(epochs):
        for i in range(n):
            idx = np.random.randint(n)
            x = X[idx]
            x = np.expand_dims(x, axis=-1)
            y = Y[idx]
            # update
            gamma = (1 / rho + np.sqrt(1 / rho**2 + 4*gamma**2)) / 2 # Theorem 2
            alpha = gamma * eta / (gamma * eta + ak**2) # Theorem 2
            ak = gamma * np.sqrt(eta*rho) # Theorem 2
            
            zeta = alpha * v + (1 - alpha) * w # equation (2)
            J_zeta = 0
            if y*(np.dot(zeta.T,x) + b) < 1: # hinge loss gradient
                J_zeta = -y*x
            v = beta * v + (1 - beta) * zeta - gamma * eta * J_zeta # equation (3)
            w = zeta - eta * J_zeta # equation (1)
            
        z = np.dot(X, w) + b
        pred = z > 0
        pred = pred * 2 - 1
        acc = np.mean(pred == Y)
        accs.append(acc)
        loss = np.mean(np.maximum(0, 1 - Y* z))
        losses.append(loss)
        
    return losses, accs

In [ ]:
tau = 0.1
acc_losses_10, acc_accs_10 = Acc_SGD(X, Y, epochs, 1 / tau, tau)
print(acc_losses_10)
print(acc_accs_10)

plt.plot(range(epochs+1), [math.log10(i)  for i in sgd_losses], "blue")
plt.plot(range(epochs+1), [math.log10(i)  for i in acc_losses_10], "black")
plt.grid()
plt.xlabel('Number of Effective Passes')
plt.ylabel('Log-Loss')
plt.legend(['SGD', 'Acc-SGD'], loc='lower left')
plt.savefig('tau_01.png')


In [ ]:
tau = 0.05
acc_losses_10, acc_accs_10 = Acc_SGD(X, Y, epochs, 1 / tau, tau)
print(acc_losses_10)
print(acc_accs_10)

plt.plot(range(epochs+1), [math.log10(i)  for i in sgd_losses], "blue")
plt.plot(range(epochs+1), [math.log10(i)  for i in acc_losses_10], "black")
plt.grid()
plt.xlabel('Number of Effective Passes')
plt.ylabel('Log-Loss')
plt.legend(['SGD', 'Acc-SGD'], loc='lower left')
plt.savefig('tau_005.png')

In [ ]:
tau = 0.01
acc_losses_10, acc_accs_10 = Acc_SGD(X, Y, epochs, 1 / tau, tau)
print(acc_losses_10)
print(acc_accs_10)

plt.plot(range(epochs+1), [math.log10(i)  for i in sgd_losses], "blue")
plt.plot(range(epochs+1), [math.log10(i)  for i in acc_losses_10], "black")
plt.grid()
plt.xlabel('Number of Effective Passes')
plt.ylabel('Log-Loss')
plt.legend(['SGD', 'Acc-SGD'], loc='lower left')
plt.savefig('tau_001.png')

In [ ]:
tau = 0.5
acc_losses_10, acc_accs_10 = Acc_SGD(X, Y, epochs, 1 / tau, tau)
print(acc_losses_10)
print(acc_accs_10)

plt.plot(range(epochs+1), [math.log10(i)  for i in sgd_losses], "blue")
plt.plot(range(epochs+1), [math.log10(i)  for i in acc_losses_10], "black")
plt.grid()
plt.xlabel('Number of Effective Passes')
plt.ylabel('Log-Loss')
plt.legend(['SGD', 'Acc-SGD'], loc='lower left')
plt.savefig('tau_05.png')